In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import *
from spot.static_analysis import UsageAnalysis, PythonProject
from spot.function_dataset import data_project_from_dir
import os

os.chdir(proj_root())

In [22]:
from coeditor.history import *

history = get_commit_history(proj_root(), 50)
for cinfo in history[:10]:
    print(cinfo.msg)

edits = edits_from_commit_history(proj_root(), history)

Implement EditSelectors.
Improve diff visualization.
Improve edit context construction.
- Bugfix: `from_code_changes` mutates original copy. - Collect only usees in editing ctx. - Improve editing visualization.
Generate contextual edits using static analysis.
Properly handle merge commits.
Construct edits incrementally from git history.
Init commit for the coeditor branch.
Rename ElemPath.
Get rid of unsued files.
Starting task: Retriving initial project from commit: c7f133ac3d92b45840424e29a099c1590004acbd
(4.8s) Finished task: Retriving initial project from commit: c7f133ac3d92b45840424e29a099c1590004acbd


Edits from commits: 100%|██████████| 49/49 [00:58<00:00,  1.19s/it]


In [70]:
from coeditor.history import *
from coeditor.encoding import *

all_mods = [c for e in edits for c in e.all_elem_changes() if isinstance(c, Modified)]
c = all_mods[0]
print(show_change(c))

* Modified: 
    def test_usage_analysis():
    ... 112 lines omitted ...
            usage1(5)  # from file2, which shadows file3
            C(5)  # from file1
            nonexist()
 
        """
 
    -       project = PythonProject.from_modules(
    -           [
    -               PythonModule.from_cst(cst.parse_module(code1), "root.file1"),
    -               PythonModule.from_cst(cst.parse_module(code2), "root.file2"),
    -               PythonModule.from_cst(cst.parse_module(code3), "root.file3"),
    -               PythonModule.from_cst(cst.parse_module(code4), "root.file4"),
    -           ]
    +       project = project_from_code(
    +           {
    +               "root.file1": code1,
    +               "root.file2": code2,
    +               "root.file3": code3,
    +               "root.file4": code4,
    +           }
            )
            analysis = UsageAnalysis(project)
 
            analysis.assert_usages(
                "root.file2/usage1",
         

In [4]:
from coeditor.history import *

n_add = n_del = n_mod = 0
for e in edits:
    for c in e.all_elem_changes():
        if isinstance(c, Added):
            n_add += 1
        elif isinstance(c, Deleted):
            n_del += 1
        elif isinstance(c, Modified):
            n_mod += 1
        else:
            raise ValueError(c)
print("n_commit:", len(edits))
print(f"n_add: {n_add}")
print(f"n_del: {n_del}")
print(f"n_mod: {n_mod}")


n_commit: 49
n_add: 339
n_del: 246
n_mod: 240


In [5]:
analyzed_edits = analyze_edits(edits)


Starting task: Performing intial module-level analysis...
(6.4s) Finished task: Performing intial module-level analysis...


Analyzing edits: 100%|██████████| 49/49 [03:52<00:00,  4.75s/it]


,name,count,avg_time,total_time
1,UsageAnalysis,98,1.579122,154.753982
2,ModuleAnlaysis/Incremental,182,0.412723,75.115604
0,ModuleAnlaysis/Initial,1,6.426628,6.426628
3,_select_change_ctx,240,0.000005,0.001288


In [27]:
selected, all_cedits = select_edits(analyzed_edits, EditSelectors.api_change_to_callsite)
coverage = set[tuple[ProjectPath, str]]()

out_file = Path("output/api_change_to_callsite.txt")
with open(out_file, "w") as f:
    for ce in selected:
        for c in ce.grouped_ctx_changes["users"]:
            coverage.add((get_change_path(c), not_none(ce.commit_info).hash))

        ce.pprint(file=f)
        print("~" * 50, "\n", file=f)

print("All modifications:", len(all_cedits))
print("User changes:", len(coverage))
print("Coverage:", f"{len(coverage) / len(all_cedits):.1%}")


All modifications: 240
User changes: 29
Coverage: 12.1%


In [6]:
selected2, all_cedits2 = select_edits(analyzed_edits, EditSelectors.usee_changes_to_user)

out_file = Path("output/pretrain.txt")
with open(out_file, "w") as f:
    for ce in selected2:
        ce.pprint(file=f)
        print("~" * 50, "\n", file=f)

print("All modifications:", len(all_cedits2))
print("User changes:", len(selected2))
print("Coverage:", f"{len(selected2) / len(all_cedits2):.1%}")


All modifications: 240
User changes: 156
Coverage: 65.0%


In [ ]:
# ==== End of new contents ====


In [ ]:
dataset = "ManyTypes4Py"

result_paths = {
    "CodeT5": get_eval_dir(dataset, ""),
    "TypeT5": get_eval_dir(
        dataset,
        "(implicit_imports, new) model-v7--TrainingConfig(drop_env_types=False, add_implicit_rel_imports=True)",
    ),
}


In [ ]:
ex_proj = PythonProject.from_root(Path("/home/jiayi/Projects/type4py"))
analysis = UsageAnalysis(
    ex_proj, add_implicit_rel_imports=True, add_override_usages=True
)
pretty_print_dict(analysis.get_stats())


In [ ]:
from spot.data import (
    create_tokenized_srcsets,
    get_tk_dataset_name,
    load_tokenized_srcsets,
    TypeCheckSettings,
)
from spot.tokenized_src import PreprocessArgs

pre_args = PreprocessArgs()
dataset = "InferTypes4Py"
sdata_name = get_tk_dataset_name(dataset, pre_args, False)
sdata_path = get_dataroot() / "TokenizedSrcSets" / sdata_name
create_tokenized_srcsets(
    dataset,
    sdata_path,
    func_only=False,
    pre_args=pre_args,
)
tk_dataset = load_tokenized_srcsets(sdata_path)
tk_dataset["test"].print_stats()


In [ ]:
from spot.static_analysis import ProjectPath

analysis.user2used[ProjectPath.from_str("type4py.type_check/MypyManager._build_tc_cmd")]


In [ ]:
from spot.utils import *


def show_subtokens(ids: list[int]):
    tks = [f"⦗{x}⦘" for x in DefaultTokenizer.convert_ids_to_tokens(ids)]
    print(" ".join(tks))


ex_code = """
def eval_on_dataset(
    model: <extra_id_0>,
    data: <extra_id_1>,
    window_size: <extra_id_2> = None,
)
"""

encoding = DefaultTokenizer.encode(ex_code)
print(encoding)
show_subtokens(encoding)


In [ ]:
ex_output = """
<extra_id_0>ModelWrapper<extra_id_1>TokenizedSrcSet<extra_id_2>Optional[int]
"""
print(decoding := DefaultTokenizer.encode(ex_output))
show_subtokens(decoding)


In [ ]:
from spot import proj_root
from spot.static_analysis import ProjectPath, UsageAnalysis, PythonProject
from pprint import pprint


proj = PythonProject.from_root(proj_root())
for caller, callees in UsageAnalysis(proj).user2used.items():
    if caller.module == "spot.static_analysis":
        print(caller)
        for callee in callees:
            print("\t", callee.used, "" if callee.is_certain else "  (maybe)")


In [ ]:
%load_ext autoreload
%autoreload 2

from spot.tokenized_src import PreprocessArgs, proj_root
from spot.function_dataset import repo_to_tk_srcs, dataset_from_repos

srcs = repo_to_tk_srcs(proj_root(), PreprocessArgs(drop_env_types=True))

from spot.data import TokenizedSrcSet, CtxArgs

sdata = TokenizedSrcSet(proj_root(), srcs)
ctx_args = CtxArgs(1024, 128, 256, 512)
cdata = sdata.to_chunks(ctx_args)

In [ ]:
from spot.utils import *

print(decode_tokens(cdata.data["input_ids"][0]))


In [ ]:
for src in [s for s in srcs if "static_analysis/PythonFunction" in str(s.file)][:10]:
    print(f"======= file: {src.file} ========")
    src.print_code(max_lines=250)


In [ ]:
from spot.data import PreprocessArgs, repr_modified_args

repr_modified_args(
    TrainingConfig(pre_args=PreprocessArgs(imports_in_preamble=False)), flatten=True
)


In [ ]:
from spot.static_analysis import cst, PythonModule, compute_module_usages, PythonProject

code1 = """
# root.file1

# global function
def gf(x):
    return x * x

# with inner function
def gf_with_inner(x):
    def inner(y):
        return y * y
    return inner(x)

# class
class C:
    def __init__(self, x):
        self.x = x
    
    def foo(self, y):
        return self.x + y

    @staticmethod
    def s_method(x):
        return x + 1
    
"""
code2 = """
# root.file2
from .file1 import gf
from root.file1 import gf_with_inner
import root.file1
import root.file1 as f1

def usage1(x):
    gf(x) + root.file1.C(5)
    foo(5)

def usage2(x):
    def inner():
        1 + gf_with_inner(x)
    return inner()

def usage_method1(x):
    x = f1.C(5)
    1 + x.foo(3)

def usage_method2(x):
    (1 + f1.C(5)).foo(3)

def usage_local():
    usage1(3)
    UsageClass(4)

@f1.C(1)
def usage_dec():
    pass

class UsageClass:
    def __init__(self, x):
        self.x = gf_with_inner(x)
        self.y = self.foo(5)

    def foo(self, y):
        return usage_local(f1.gf(y))

    @staticmethod
    def s_method(x):
        return x

class SubClass(UsageClass):
    def use(self):
        self.foo(5)
        f1.C.s_method(5)
"""

code3 = """
# root.file3
from .file1 import *

def usage1(x):
    gf(5)
    C(5)
    
"""

project = PythonProject.from_modules(
    [
        PythonModule.from_cst(cst.parse_module(code1), "root.file1"),
        PythonModule.from_cst(cst.parse_module(code2), "root.file2"),
        PythonModule.from_cst(cst.parse_module(code3), "root.file3"),
    ]
)

for u in compute_module_usages(project.modules["root.file3"]):
    print(str(u))


In [ ]:
from spot.static_analysis import build_project_namespaces

build_project_namespaces(project)


In [ ]:
from spot.static_analysis import compute_module_usages

compute_module_usages(project.modules["root.file3"])


In [ ]:
from spot.static_analysis import UsageAnalysis, build_project_namespaces

build_project_namespaces(project)

analysis = UsageAnalysis(project)
analysis.caller2callees[ProjectPath("root.file2", "SubClass.use")]


In [ ]:
import libcst as cst

from spot.tokenized_src import TokenizedSrc, PreprocessArgs
from spot.utils import Path, decode_tokens

ex_code = '''# document comment 1
  # document comment 2
"""String document commnet"""
import os; import spot;
from sys import argv, exit
# after import
@wraps(function)
def catch_permission_denied(function):
    import some.inner.imports
    """
    Decorator to catch :class:`psycopg2.ProgrammingError` exceptions with the
    ``INSUFFICIENT_PRIVILEGE`` error code and rethrow them as
    :class:`~werkzeug.exceptions.Forbidden` exceptions instead.
    """
    @wraps(function)
    def decorated(x: str, y: int) -> str:
        try:
            # comment 1
            # comment 1 cont
            return function(*args, **kwargs)

        except InsufficientPrivilege as error:
            LOG.error("Forbidden: %s", error) # comment 2
            raise Forbidden()

    return decorated
'''
pre_args = PreprocessArgs(stub_in_preamble=True)
ex_src = TokenizedSrc.parse(ex_code, Path("test_file"), Path("test_repo"), pre_args)
print(decode_tokens(ex_src.tokenized_code))


In [ ]:
from spot.data import src_to_chunks_, CtxArgs, PreprocessArgs
from ipywidgets import interactive

pre_args = PreprocessArgs(stub_in_preamble=True)
ex_src = TokenizedSrc.parse(ex_code, Path("test_file"), Path("test_repo"), pre_args)


def print_code(
    preamble: int,
    left: int,
    right: int,
    ctx_size: int,
    max_labels: int,
    chunk_id: int,
    inline_prev: bool,
):
    chunks = []
    args = CtxArgs(
        ctx_size,
        preamble,
        left,
        right,
        max_labels=max_labels,
        inline_prev_gold=inline_prev,
    )
    src_to_chunks_(chunks, [], ex_src, (0, len(ex_src.types)), args)
    print(decode_tokens(chunks[chunk_id]["input_ids"]))


interactive(
    print_code,
    preamble=(1, 100),
    left=(1, 200),
    right=(1, 100),
    ctx_size=(1, 500),
    max_labels=(1, 10),
    chunk_id=(0, 1),
    inline_prev=True,
)


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from typing import *

import pandas as pd
import plotly.express as px

from spot.data import GitRepo, ModuleRemapUnpickler
from spot.type_env import (
    AnnotPath,
    MypyChecker,
    SelectAnnotations,
    TypeInfAction,
    TypeInfEnv,
    TypeInfState,
    mypy_checker,
)
from spot.utils import cst, proj_root, read_file, seq_flatten, tqdm, write_file

os.chdir(proj_root())

datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"

useful_repos_path = proj_root() / "scripts" / "useful_repos.pkl"
rename_module = lambda n: "spot.data" if n == "spot.data_prepare" else n
with useful_repos_path.open("rb") as f:
    useful_repos: list[GitRepo] = ModuleRemapUnpickler(f, rename_module).load()

In [ ]:
# loading pre-trained model and tokenizer
from spot.utils import get_data_dir

model_dir = "Salesforce/codet5-base"
# model_dir = datadir / "checkpoints/saved/SPOT-CodeT5-no_margin/"

import torch
from transformers import (
    DataCollatorForSeq2Seq,
    RobertaTokenizer,
    T5ForConditionalGeneration,
)
from transformers.models.t5 import T5ForConditionalGeneration

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer: RobertaTokenizer = RobertaTokenizer.from_pretrained(model_dir)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained(
    model_dir
).to(device)
max_target_length = 128


In [ ]:
from spot.data import mask_type_annots, output_ids_as_types, tokenize_masked

test_code = """
@dataclass
class GitRepo:
    author: str
    name: str
    url: str
    stars: int
    forks: int

    def authorname(self):
        return self.author + "__" + self.name

    def repo_dir(self, repos_dir: Path) -> Path:
        return repos_dir / "downloaded" / self.authorname()

    def download(self, repos_dir: Path, timeout=None) -> bool:
        pass
"""


def run_model(code: str, num_beams=16):
    masked = mask_type_annots((Path("no_source"), code))
    tks = tokenize_masked(masked, tokenizer, device)
    input_ids = tks["input_ids"]
    with torch.no_grad():
        loss = model.forward(**tks).loss
        dec = model.generate(
            input_ids,
            max_length=max_target_length,
            num_beams=num_beams,
            # do_sample=True,
        )[0]
    return {
        "loss": loss,
        "predicted_types": output_ids_as_types(dec, tokenizer),
        "labels": output_ids_as_types(tks["labels"][0], tokenizer),
        "generation": tokenizer.decode(dec),
        "input_ids": input_ids[0],
        "output_ids": dec,
        "annots_info": masked["annots_info"],
    }


result = run_model(test_code, num_beams=10)
result["loss"]


In [ ]:
from spot import PythonType
from spot.type_env import apply_annotations


def type_to_annot(ty: PythonType) -> str:
    return cst.Annotation(cst.parse_expression(str(ty)))


def run_aug_model(src: Path, cwd: Path):
    result = run_model(read_file(src), num_beams=10)
    pred_annots = {
        info.path: type_to_annot(t)
        for info, t in zip(result["annots_info"], result["predicted_types"])
    }
    m1 = apply_annotations(cst.parse_module(read_file(src)), pred_annots)
    write_file(src, m1.code)
    checker_r = MypyChecker.check_project(src, cwd)
    pos_to_preds = {
        info.annot_range: str(ty)
        for info, ty in zip(result["annots_info"], result["predicted_types"])
    }
    return {
        "model_result": result,
        "module": m1,
        "checker_feedback": checker_r,
        "pos_to_preds": pos_to_preds,
    }


aug_r = run_aug_model(inference_dir / "env_code_2.py", inference_dir)


In [ ]:
from spot.utils import patch_code_with_extra

print("---- predicted types ----")
print(aug_r["model_result"]["predicted_types"])
print("---- model output ----")
print(tokenizer.decode(aug_r["model_result"]["output_ids"], skip_special_tokens=False))
print("---- checker_feedback ----")
print(aug_r["checker_feedback"].output_str)

print("---- new input ----")
new_input = patch_code_with_extra(
    aug_r["module"].code,
    aug_r["pos_to_preds"],
    aug_r["checker_feedback"].error_dict["env_code_2.py"],
)
print(new_input)


In [ ]:
import pickle

from spot.utils import Path, run_long_task, DefaultTokenizer, not_none, CountedAcc
from spot import proj_root
from spot.function_dataset import guess_src_root

datadir = Path(not_none(os.getenv("datadir")))
repos_dir = datadir / "SPOT-data/repos/"

repos_split_path = proj_root() / "data/repos_split.pkl"
with repos_split_path.open("rb") as f:
    repos_split = pickle.load(f)

root_is_src = list[bool]()
for repo in repos_split["train"]:
    rd = repo.repo_dir(repos_dir)
    root_is_src.append(guess_src_root(rd).name == "src")

CountedAcc(sum(root_is_src), len(root_is_src))


In [ ]:
src_in_root = 0
package_in_root = 0
setup_in_root = 0
n_proj = 0

weird_repos = []
setup_files = []

for repo in repos_split["train"]:
    rd: Path = repo.repo_dir(repos_dir)
    n_proj += 1
    files = list(rd.iterdir())
    if rd / "src" in files:
        src_in_root += 1
    elif rd / (pname := rd.name.split("__")[-1]) in files:
        package_in_root += 1
    elif rd / "setup.cfg" in files:
        setup_in_root += 1
        setup_files.append(rd / "setup.cfg")
    else:
        weird_repos.append(repo)

print("n_projects:", n_proj)
print("src_in_root:", src_in_root)
print("package_in_root:", package_in_root)
print("setup_in_root:", setup_in_root)
print("weird_repos:", len(weird_repos))


In [ ]:
for repo in weird_repos[:10]:
    rd: Path = repo.repo_dir(repos_dir)
    print("Repo:", rd.relative_to(repos_dir))
    for f in rd.iterdir():
        print(f.relative_to(rd))
